In [1]:
import os

In [2]:
%pwd

'e:\\PROJECTS\\Loan_Approval_Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\PROJECTS\\Loan_Approval_Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import LogisticRegression
import joblib
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()

        X_train_scaled = scaler.fit_transform(train_x)
        X_test_scaled = scaler.transform(test_x)

        model = LogisticRegression()
        model.fit(X_train_scaled, train_y)

        print("Validation Accuracy:", model.score(X_test_scaled, test_y))

        # Save model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

        # ✅ Save scaler
        joblib.dump(scaler, os.path.join(self.config.root_dir, "scaler.pkl"))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-08-31 20:55:06,244: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-31 20:55:06,248: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-31 20:55:06,250: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-31 20:55:06,250: INFO: common: created directory at: artifacts]
[2025-08-31 20:55:06,256: INFO: common: created directory at: artifacts/model_trainer]
Validation Accuracy: 0.9026217228464419


e:\PROJECTS\Loan_Approval_Prediction\myenv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
